In [34]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/DavidIbrahimG/Project1.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "DavidIbrahimG"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "57784cd3735838b36046f55be85608fe82144f40"

In [2]:
## checking what PWD is
%pwd

'/Users/davidibrahim/Programming/mlops1/Project1/research'

In [3]:
## changing to the project directory
%cd ..

/Users/davidibrahim/Programming/mlops1/Project1


/Users/davidibrahim/Programming/mlops1/Project1/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
## checking our current directory
%pwd

'/Users/davidibrahim/Programming/mlops1/Project1'

In [35]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [36]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

In [41]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(str(config_filepath))
        self.params = read_yaml(str(params_filepath))
        self.schema = read_yaml(str(schema_filepath))

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> DataIngestionConfig:

        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri = "https://dagshub.com/DavidIbrahimG/Project1.mlflow"
        )
        return model_evaluation_config

In [42]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [53]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, predicted):
        rmse = np.sqrt(mean_squared_error(actual, predicted))
        mae = mean_absolute_error(actual, predicted)
        r2 = r2_score(actual, predicted)

        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run() as run:

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            ## saving metrics as local 
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            ## logging params in MLflow
            mlflow.log_params(self.config.all_params)

            ## logging metrics in MLflow
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            ## model registry does not work with file store
            """if tracking_uri_type_store != "file":
                # Log the model to the remote server and register it
                mlflow.sklearn.log_model(model, "model", registered_model_name="model")
            else:
                # Log the model to the local file system (no model registration)
                mlflow.sklearn.log_model(model, "model")"""


In [54]:
## try catch to execute the code
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

[2025-07-06 00:37:27,137 - datascience - INFO - YAML file config/config.yaml loaded successfully.]
[2025-07-06 00:37:27,138 - datascience - INFO - YAML file params.yaml loaded successfully.]
[2025-07-06 00:37:27,141 - datascience - INFO - YAML file schema.yaml loaded successfully.]
[2025-07-06 00:37:27,142 - datascience - INFO - Directory artifacts created or already exists.]
[2025-07-06 00:37:27,142 - datascience - INFO - Directory artifacts/model_evaluation created or already exists.]
[2025-07-06 00:37:27,364 - datascience - INFO - Data saved to JSON file artifacts/model_evaluation/metrics.json.]
🏃 View run valuable-kite-26 at: https://dagshub.com/DavidIbrahimG/Project1.mlflow/#/experiments/0/runs/1bdb66b124a94e4bb5f1ad7298feb451
🧪 View experiment at: https://dagshub.com/DavidIbrahimG/Project1.mlflow/#/experiments/0
